# Final Project
*Submission deadline: December 9, 2019, 11:59PM*

**The final grading is based on:**
 - Functionality of your algorithm
 - GUI design
 - Rendering result
 - You need to include at least 3 images and the rendering results from your algorithm
 
**You will receive extra points if:**
 - Output is outstanding
 - Creative ideas
 - Interesting combinations or add-ons
 - Implementation is challenging
 
Please follow the homework submission guidelines. Be sure to zip all the necessary files including codes, images, and a project report. **In your project report, you should specify:**
 - Effects/functions you want to achieve
 - Details of the algorithm and implementation
 - Instruction on how to run all functionalities of your program
 - Comments on what you have learned and new features that can be added in the future.

In [19]:
from tkinter import filedialog
import tkinter as tk
import cv2
import PIL.Image, PIL.ImageTk
import numpy as np

# global variables
MARGIN = 10  # px
MAXDIM = 580

class App():
   
    def __init__(self, window, window_title, image_path="lena.bmp"):## paths for 3 images are: iu.jpg, iu2.jpg, lena.bmp
        self.window = window
        self.window.title(window_title)
        self.neg = False
        self.histEq = False
        self.pencil_switch = False
        
        # Load an image using OpenCV
        self.cv_img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        
        # Get the image dimensions (OpenCV stores image data as NumPy ndarray)
        self.height, self.width, no_channels = self.cv_img.shape
        
        ''' Image Display Related Code'''
        # Create a FRAME that can fit the images
        self.frame1 = tk.Frame(self.window, width=self.width, height=self.height, bg='cornsilk2')
        self.frame1.pack(fill=tk.BOTH)
        
        # Create a CANVAS for original image
        self.canvas0 = tk.Canvas(self.frame1, width=MAXDIM, height=MAXDIM+(3*MARGIN), bg='cornsilk2')
        self.canvas0.pack(side=tk.LEFT)
        
        # Create a CANVAS for changing image
        self.canvas1 = tk.Canvas(self.frame1, width=MAXDIM, height=MAXDIM+(3*MARGIN), bg='cornsilk2')
        self.canvas1.pack(side=tk.LEFT)
        
        # Use PIL (Pillow) to convert the NumPy ndarray to a PhotoImage
        self.photoOG = PIL.ImageTk.PhotoImage(image=PIL.Image.fromarray(self.cv_img))
        self.photo = PIL.ImageTk.PhotoImage(image=PIL.Image.fromarray(self.cv_img))
        
        # Add a PhotoImage to the Canvas (original)
        self.canvas0.create_image(MAXDIM//2, MAXDIM//2, image=self.photoOG)
        
        # Add a PhotoImage to the Canvas (changing effects)
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        
        # Write labels for both images, font/size can be changed
        self.canvas0.create_text(MAXDIM//2, MAXDIM+(2*MARGIN),font="Tahoma 20",text="Original Photo")
        self.canvas1.create_text(MAXDIM//2, MAXDIM+(2*MARGIN),font="Tahoma 20",text="Modified Photo")
        
# ##############################################################################################
# ################################   PARAMETER TOOLBAR   #######################################
# ##############################################################################################

        # Create a FRAME that can fit the features
        self.frame2 = tk.Frame(self.window, width=self.width, height=self.height//2, bg='cornsilk3')
        self.frame2.pack(side=tk.BOTTOM, expand=1, fill=tk.X)
        
        # GUI Decription Text
        self.label_og = tk.Label(self.frame2, text="Photo Editor", font="Tahoma 20 bold")
        self.label_og.pack(anchor=tk.W)
        self.label_og = tk.Label(self.frame2, font="Tahoma 16", justify=tk.LEFT,
                text="This is the photo editor by Jara! :-)")
        self.label_og.pack(anchor=tk.W)
        
        # Create a CANVAS for buttons
        self.canvas_but = tk.Canvas(self.frame2, bd=2)
        self.canvas_but.pack(side=tk.LEFT, anchor=tk.W)
        
        '''#################################  BUTTONS  ###############################'''
        # Create a BUTTON that negates the image
        self.btn_neg = tk.Button(self.canvas_but, text="Negate", font="Tahoma 10", command=self.negate)
        self.btn_neg.pack(anchor=tk.CENTER)
        
        self.btn_reset = tk.Button(self.canvas_but, text="Reset", font="Tahoma 10", command=self.reset)
        self.btn_reset.pack(anchor=tk.CENTER)
        
        self.btn_HE = tk.Button(self.canvas_but, text="auto-adjust", font="Tahoma 10", command=self.hist_eq_switch)
        self.btn_HE.pack(anchor=tk.CENTER)
        
        self.btn_pencil = tk.Button(self.canvas_but, text="Pencil Effect", font="Tahoma 10", command=self.pencil_effect)
        self.btn_pencil.pack(anchor=tk.CENTER)
        
        
        '''#################################  SLIDERS  ###############################'''
         # Create a SCALE that lets the user blur the image
        self.scl_blur = tk.Scale(self.frame2, from_=1, to=100, orient=tk.HORIZONTAL, showvalue=1, 
                                 command = self.blur_image, sliderlength=30, label="Blur", font="Tahoma 12")
        self.scl_blur.pack(side=tk.RIGHT, anchor=tk.W)
        
        self.scl_bilateral = tk.Scale(self.frame2, from_=1, to=100, orient=tk.HORIZONTAL, showvalue=1, 
                                 command = self.bilateral_KeepEdge, sliderlength=30, label="Cartoon Effect", font="Tahoma 12")
        self.scl_bilateral.pack(side=tk.RIGHT, anchor=tk.W)
        

        self.scl_R = tk.Scale(self.frame2, from_=0.00, to=2.99, resolution = 0.01, orient=tk.HORIZONTAL, showvalue=1, 
                                 command = self.red, sliderlength=30, label="R", font="Tahoma 12")
        self.scl_R.set(2.0)
        self.scl_R.pack(side=tk.RIGHT, anchor=tk.W)
        

        self.scl_G = tk.Scale(self.frame2, from_=0.00, to=2.99, resolution = 0.01, orient=tk.HORIZONTAL, showvalue=1, 
                                 command = self.green, sliderlength=30, label="G", font="Tahoma 12")
        self.scl_G.set(2.0)
        self.scl_G.pack(side=tk.RIGHT, anchor=tk.W)
        
        self.scl_B = tk.Scale(self.frame2, from_=0.00, to=2.99, resolution = 0.01, orient=tk.HORIZONTAL, showvalue=1, 
                                 command = self.blue, sliderlength=30, label="B", font="Tahoma 12")
        self.scl_B.set(2.0)
        self.scl_B.pack(side=tk.RIGHT, anchor=tk.W)
        
        self.scl_corr = tk.Scale(self.frame2, from_=0.00, to=2.99, resolution = 0.01, orient=tk.HORIZONTAL, showvalue=1, 
                                 command = self.gamma_corr_slider, sliderlength=30, label="Gamma Correction", font="Tahoma 12")
        self.scl_corr.set(2.0)
        self.scl_corr.pack(side=tk.RIGHT, anchor=tk.W)
        
        self.window.mainloop()

##############################################################################################
#################################  CALLBACK FUNCTIONS  #######################################
##############################################################################################
    '''#################################  reset button  ###############################'''
    def reset(self):
        self.scl_B.set(2.0)
        self.scl_G.set(2.0)
        self.scl_R.set(2.0)
        self.scl_corr.set(2.0)
        self.scl_blur.set(1)
        self.scl_bilateral.set(1)
        self.neg = False
        self.histEq = False
        self.pencil_switch = False
        
        
        temp = self.cv_img.copy()
        
        if (self.neg):
            temp = 255 - temp
        
        if (self.histEq):
            temp = self.hist_equalization(temp)
            
        
        temp[:,:, 0] = self.gammaCorr_byColor(temp[:,:, 0], 3.00 - self.scl_R.get(), 256)
        temp[:,:, 1] = self.gammaCorr_byColor(temp[:,:, 1], 3.00 - self.scl_G.get(), 256)
        temp[:,:, 2] = self.gammaCorr_byColor(temp[:,:, 2], 3.00 - self.scl_B.get(), 256)
        
        
        temp = self.gammaCorr(temp, 3.0 - self.scl_corr.get(), 256)
        temp = cv2.blur(temp, (self.scl_blur.get(), self.scl_blur.get()))
        temp = cv2.bilateralFilter(temp, -1, self.scl_bilateral.get(), 1)
        
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(temp))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        
    '''#################################  generate PDF ###############################'''
    def generatePDF(self, size, channel):
        pdf = np.zeros(size)

        row = len(channel)
        col = len(channel[0])

        for i in range(row):
            for j in range(col):
                pdf[channel[i, j]] = pdf[channel[i, j]] + 1

        pdf = pdf.astype(int)
        return pdf

    '''#################################  generate CDF ###############################'''
    def generateCDFfromPDF(self, size, pdf):
        sum = 0
        cdf = np.zeros(size)
        for i in range(size):
            sum = sum + pdf[i]
            cdf[i] = sum
        return cdf

    '''#################################  create histogram equalization map ###############################'''
    def hist_eq (self, cdf, col, row, Range=256):
        HEmap = cdf * (Range - 1) / (col * row)
        HEmap = HEmap.astype('uint8')
        return HEmap
    
    '''#################################  histogram equalization ###############################'''
    def hist_equalization(self, img):
        self.histEq = not self.histEq
        
        HSVimg = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        h, s, v = cv2.split(HSVimg)
        pdf = self.generatePDF(256, v)
        cdf = self.generateCDFfromPDF(256, pdf)
        
        HEmap = self.hist_eq(cdf, v.shape[0], v.shape[1])
        Val_HE = np.zeros(v.shape, dtype='uint8')
        for i in range(len(Val_HE)):
            for j in range(len(Val_HE[0])):
                Val_HE[i, j] = HEmap[v[i, j]]

        # convert back to bgr image
        hsv_HE = cv2.merge([h, s, Val_HE])
        rgb_HE = cv2.cvtColor(hsv_HE, cv2.COLOR_HSV2RGB)
        return rgb_HE
    
    
    '''#################################  gammar corr ###############################'''
    def gammaCorr(self, img, factor, size):
        temp = img.copy()
        temp[:,:,0] = self.gammaCorr_byColor(img[:,:,0], factor, size)
        temp[:,:,1] = self.gammaCorr_byColor(img[:,:,1], factor, size)
        temp[:,:,2] = self.gammaCorr_byColor(img[:,:,2], factor, size)
        return temp
        
    
    '''#################################  gammar corr by color ###############################'''
    def gammaCorr_byColor(self, channel, factor, size):
        if (factor != 1):
            new = channel / (size - 1)
            new = np.power(new, factor)
            new = (new * (size - 1)).astype(int)
            return new
        else:
            return channel
    
    
    
    
    
    
    
    
    
    
    def pencil_effect(self):
        self.pencil_switch = not self.pencil_switch
        
        temp = self.cv_img.copy()
        if (self.histEq):
            temp = self.hist_equalization(temp)
            
        if (self.neg):
            temp = 255 - temp
        
        temp = self.gammaCorr(temp, 3.0 - self.scl_corr.get(), 256)
        temp[:,:, 0] = self.gammaCorr_byColor(temp[:,:, 0], 3.00 - self.scl_R.get(), 256)
        temp[:,:, 1] = self.gammaCorr_byColor(temp[:,:, 1], 3.00 - self.scl_G.get(), 256)
        temp[:,:, 2] = self.gammaCorr_byColor(temp[:,:, 2], 3.00 - self.scl_B.get(), 256)
        temp = cv2.blur(temp, (self.scl_blur.get(), self.scl_blur.get()))
        temp = cv2.bilateralFilter(temp, -1, self.scl_bilateral.get(), 4)
        
        
        if (self.pencil_switch):
            temp_gray = cv2.cvtColor(temp, cv2.COLOR_RGB2GRAY)
            temp_blur = cv2.GaussianBlur(temp_gray, (21, 21), 0)
            result = cv2.divide(temp_gray, temp_blur, scale=256)
        else: 
            result = temp
        
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(result.astype('uint8')))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
    
    def hist_eq_switch(self): 
        self.histEq = not self.histEq
        ## main effect
        temp = self.cv_img.copy()
        if (self.histEq):
            temp = self.hist_equalization(temp)
            
        if (self.neg):
            temp = 255 - temp
        
        temp = self.gammaCorr(temp, 3.0 - self.scl_corr.get(), 256)
        temp[:,:, 0] = self.gammaCorr_byColor(temp[:,:, 0], 3.00 - self.scl_R.get(), 256)
        temp[:,:, 1] = self.gammaCorr_byColor(temp[:,:, 1], 3.00 - self.scl_G.get(), 256)
        temp[:,:, 2] = self.gammaCorr_byColor(temp[:,:, 2], 3.00 - self.scl_B.get(), 256)
        temp = cv2.blur(temp, (self.scl_blur.get(), self.scl_blur.get()))
        temp = cv2.bilateralFilter(temp, -1, self.scl_bilateral.get(), 4)
        
        if (self.pencil_switch):
            temp_gray = cv2.cvtColor(temp, cv2.COLOR_RGB2GRAY)
            temp_inv = 255 - temp_gray
            temp_inv_blur = cv2.GaussianBlur(temp_inv, (21, 21), 0)
            temp = cv2.divide(temp_gray, 255-temp_inv_blur, scale=256)
        
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(temp))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        
    
    '''#################################  gamma correction adjust  ###############################'''
    ### Callback for negate button
    def gamma_corr_slider(self, k):
        temp = self.cv_img.copy()
        temp = self.gammaCorr(temp, 3.0 - self.scl_corr.get(), 256)
        
        temp[:,:, 0] = self.gammaCorr_byColor(temp[:,:, 0], 3.00 - self.scl_R.get(), 256)
        temp[:,:, 1] = self.gammaCorr_byColor(temp[:,:, 1], 3.00 - self.scl_G.get(), 256)
        temp[:,:, 2] = self.gammaCorr_byColor(temp[:,:, 2], 3.00 - self.scl_B.get(), 256)
        ## main effect
        if (self.neg):
            temp = 255 - temp
        
        if (self.histEq):
            temp = self.hist_equalization(temp)
            
        temp = cv2.blur(temp, (self.scl_blur.get(), self.scl_blur.get()))
        temp = cv2.bilateralFilter(temp, -1, self.scl_bilateral.get(), 4)
        
        if (self.pencil_switch):
            temp_gray = cv2.cvtColor(temp, cv2.COLOR_RGB2GRAY)
            temp_inv = 255 - temp_gray
            temp_inv_blur = cv2.GaussianBlur(temp_inv, (21, 21), 0)
            temp = cv2.divide(temp_gray, 255-temp_inv_blur, scale=256)
        
        
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(temp))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
    
    '''#################################  Red adjust  ###############################'''
    ### Callback for negate button
    def red(self, k): 
        temp = self.cv_img.copy()
        
        temp = self.gammaCorr(temp, 3.0 - self.scl_corr.get(), 256)
        temp[:,:, 0] = self.gammaCorr_byColor(temp[:,:, 0], 3.00 - self.scl_R.get(), 256)
        temp[:,:, 1] = self.gammaCorr_byColor(temp[:,:, 1], 3.00 - self.scl_G.get(), 256)
        temp[:,:, 2] = self.gammaCorr_byColor(temp[:,:, 2], 3.00 - self.scl_B.get(), 256)
        ## main effect
        if (self.neg):
            temp = 255 - temp
            
        if (self.histEq):
            temp = self.hist_equalization(temp)
            
        temp = cv2.blur(temp, (self.scl_blur.get(), self.scl_blur.get()))
        temp = cv2.bilateralFilter(temp, -1, self.scl_bilateral.get(), 4)
        
        if (self.pencil_switch):
            temp_gray = cv2.cvtColor(temp, cv2.COLOR_RGB2GRAY)
            temp_inv = 255 - temp_gray
            temp_inv_blur = cv2.GaussianBlur(temp_inv, (21, 21), 0)
            temp = cv2.divide(temp_gray, 255-temp_inv_blur, scale=256)
        
        
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(temp))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        
    '''#################################  Green adjust  ###############################'''
    ### Callback for negate button
    def green(self, k): 
        temp = self.cv_img.copy()
        temp = self.gammaCorr(temp, 3.0 - self.scl_corr.get(), 256)
        temp[:,:, 0] = self.gammaCorr_byColor(temp[:,:, 0], 3.00 - self.scl_R.get(), 256)
        temp[:,:, 1] = self.gammaCorr_byColor(temp[:,:, 1], 3.00 - self.scl_G.get(), 256)
        temp[:,:, 2] = self.gammaCorr_byColor(temp[:,:, 2], 3.00 - self.scl_B.get(), 256)
        
        ## main effect
        if (self.neg):
            temp = 255 - temp
        
        if (self.histEq):
            temp = self.hist_equalization(temp)
            
        temp = cv2.blur(temp, (self.scl_blur.get(), self.scl_blur.get()))
        temp = cv2.bilateralFilter(temp, -1, self.scl_bilateral.get(), 4)
        
        if (self.pencil_switch):
            temp_gray = cv2.cvtColor(temp, cv2.COLOR_RGB2GRAY)
            temp_inv = 255 - temp_gray
            temp_inv_blur = cv2.GaussianBlur(temp_inv, (21, 21), 0)
            temp = cv2.divide(temp_gray, 255-temp_inv_blur, scale=256)
        
        
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(temp))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        
    '''#################################  Blue adjust  ###############################'''
    ### Callback for negate button
    def blue(self, k): 
        temp = self.cv_img.copy()
        temp = self.gammaCorr(temp, 3.0 - self.scl_corr.get(), 256)
        temp[:,:, 0] = self.gammaCorr_byColor(temp[:,:, 0], 3.00 - self.scl_R.get(), 256)
        temp[:,:, 1] = self.gammaCorr_byColor(temp[:,:, 1], 3.00 - self.scl_G.get(), 256)
        temp[:,:, 2] = self.gammaCorr_byColor(temp[:,:, 2], 3.00 - self.scl_B.get(), 256)
        
        ## main effect
        if (self.neg):
            temp = 255 - temp
            
        if (self.histEq):
            temp = self.hist_equalization(temp)
            
        temp = cv2.blur(temp, (self.scl_blur.get(), self.scl_blur.get()))
        temp = cv2.bilateralFilter(temp, -1, self.scl_bilateral.get(), 4)
        
        if (self.pencil_switch):
            temp_gray = cv2.cvtColor(temp, cv2.COLOR_RGB2GRAY)
            temp_inv = 255 - temp_gray
            temp_inv_blur = cv2.GaussianBlur(temp_inv, (21, 21), 0)
            temp = cv2.divide(temp_gray, 255-temp_inv_blur, scale=256)
        
        
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(temp))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        
    '''#################################  NEGATIVE  ###############################'''
    ### Callback for negate button
    def negate(self): 
        self.neg = not self.neg
        ## main effect
        if (self.neg):
            temp = 255 - self.cv_img
        else:
            temp = self.cv_img.copy()
        
        if (self.histEq):
            temp = self.hist_equalization(temp)
            
        
        temp = self.gammaCorr(temp, 3.0 - self.scl_corr.get(), 256)
        temp[:,:, 0] = self.gammaCorr_byColor(temp[:,:, 0], 3.00 - self.scl_R.get(), 256)
        temp[:,:, 1] = self.gammaCorr_byColor(temp[:,:, 1], 3.00 - self.scl_G.get(), 256)
        temp[:,:, 2] = self.gammaCorr_byColor(temp[:,:, 2], 3.00 - self.scl_B.get(), 256)
        temp = cv2.blur(temp, (self.scl_blur.get(), self.scl_blur.get()))
        temp = cv2.bilateralFilter(temp, -1, self.scl_bilateral.get(), 4)
        
        if (self.pencil_switch):
            temp_gray = cv2.cvtColor(temp, cv2.COLOR_RGB2GRAY)
            temp_inv = 255 - temp_gray
            temp_inv_blur = cv2.GaussianBlur(temp_inv, (21, 21), 0)
            temp = cv2.divide(temp_gray, 255-temp_inv_blur, scale=256)
        
        
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(temp))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        
    '''#################################  BLUR  ###############################'''
    # Callback for the "Blur" Scale 
    def blur_image(self, k):
        k = self.scl_blur.get()  # get value from the corresponding scale
        temp = cv2.blur(self.cv_img, (k, k))
        
        temp = self.gammaCorr(temp, 3.0 - self.scl_corr.get(), 256)
        temp[:,:, 0] = self.gammaCorr_byColor(temp[:,:, 0], 3.00 - self.scl_R.get(), 256)
        temp[:,:, 1] = self.gammaCorr_byColor(temp[:,:, 1], 3.00 - self.scl_G.get(), 256)
        temp[:,:, 2] = self.gammaCorr_byColor(temp[:,:, 2], 3.00 - self.scl_B.get(), 256)
        temp = cv2.bilateralFilter(temp, -1, self.scl_bilateral.get(), 4)
        if (self.neg):
            temp = 255 - temp
        
        if (self.histEq):
            temp = self.hist_equalization(temp)
            
        if (self.pencil_switch):
            temp_gray = cv2.cvtColor(temp, cv2.COLOR_RGB2GRAY)
            temp_inv = 255 - temp_gray
            temp_inv_blur = cv2.GaussianBlur(temp_inv, (21, 21), 0)
            temp = cv2.divide(temp_gray, 255-temp_inv_blur, scale=256)
        
            
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(temp))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        
    '''#################################  Keeping edge blur  ###############################'''
    def bilateral_KeepEdge(self, k):
        k = self.scl_bilateral.get()  # get value from the corresponding scale
        temp = cv2.bilateralFilter(self.cv_img, -1, k, 4)
        
        temp = self.gammaCorr(temp, 3.0 - self.scl_corr.get(), 256)
        temp[:,:, 0] = self.gammaCorr_byColor(temp[:,:, 0], 3.00 - self.scl_R.get(), 256)
        temp[:,:, 1] = self.gammaCorr_byColor(temp[:,:, 1], 3.00 - self.scl_G.get(), 256)
        temp[:,:, 2] = self.gammaCorr_byColor(temp[:,:, 2], 3.00 - self.scl_B.get(), 256)
        temp = cv2.blur(temp, (self.scl_blur.get(), self.scl_blur.get()))
        if (self.neg):
            temp = 255 - temp
        
        if (self.histEq):
            temp = self.hist_equalization(temp)
            
        if (self.pencil_switch):
            temp_gray = cv2.cvtColor(temp, cv2.COLOR_RGB2GRAY)
            temp_inv = 255 - temp_gray
            temp_inv_blur = cv2.GaussianBlur(temp_inv, (21, 21), 0)
            temp = cv2.divide(temp_gray, 255-temp_inv_blur, scale=256)
        
            
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(temp))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
    
    
##############################################################################################
# Create a window and pass it to the Application object
App(tk.Tk(), "440 Final Project")